## Novel Materials Design with Data-Driven Quantum Computing

Designing novel materials using the available knowledge has been a holy-grail of materials science research. Quantum algorithms promise an efficeint search in the insanely vast chemical space to find novel materials with targeted properties. In this project, leveraging an open-source dataset, we aim to design compounds with enhanced critical temperatures to guide experimental exploration.

## Prediction of Critical Temperature (Tc) of Superconductors using Quantum Machine Learning 

As a first step, we buid a predictive model which would be our surrogate model for the desired property while exploring the chemical space.

## Dataset Loading

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load datasets
train_data = pd.read_csv('/home/ayush/Downloads/superconductivty+data/train.csv')
unique_m_data = pd.read_csv('/home/ayush/Downloads/superconductivty+data//unique_m.csv')

In [ ]:
unique_m_data_subset = unique_m_data.iloc[:, :87]
merged_data = pd.concat([unique_m_data_subset.reset_index(drop=True), train_data.reset_index(drop=True)], axis=1)


In [ ]:
# Extract features and target
X = merged_data.iloc[:, :-1].values  
y = merged_data.iloc[:, -1].values   

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Dimensionality Reduction with PCA (optional)

In [ ]:
from sklearn.decomposition import PCA

# Reduce to a smaller number of features
pca = PCA(n_components=10)  # Choose a smaller number based on your needs
X_train_reduced = pca.fit_transform(X_train)
X_test_reduced = pca.transform(X_test)

## Quantum Support Vector Regression with Fidelity Quantum Kernel and ZZFeaturemap

In [ ]:
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorEstimator

# Define a feature map for Quantum Kernel Estimation
feature_map = ZZFeatureMap(feature_dimension=X_train_reduced.shape[1], reps=1, entanglement='full')

quantum_kernel = FidelityQuantumKernel(feature_map=feature_map)


In [ ]:
from qiskit_machine_learning.algorithms import QSVR

# Initialize and train the Quantum Support Vector Regressor with the quantum kernel
qsvr = QSVR(quantum_kernel=quantum_kernel)
qsvr.fit(X_train_reduced, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Predictions
y_pred_train = qsvr.predict(X_train_reduced)
y_pred_test = qsvr.predict(X_test_reduced)

# Evaluation metrics
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Quantum SVR Train MSE:", mse_train)
print("Quantum SVR Test MSE:", mse_test)
print("Quantum SVR Test MAE:", mae_test)
print("Quantum SVR Test R2 Score:", r2_test)

## Variational Quantum Regression wth ZZFeatureMap

In [ ]:
num_qubits = X_train_reduced.shape[1]
theta = ParameterVector('theta', length=num_qubits)  # Unique parameter names for feature map
# feature_map = EfficientSU2(num_qubits=num_qubits, reps=1, entanglement='linear', parameter_prefix="theta")
feature_map_new = ZZFeatureMap(feature_dimension=X_train_reduced.shape[1], reps=1, entanglement='full')


# Step 2: Set up the Variational Quantum Regressor (VQR) with COBYLA optimizer
vqr = VQR(feature_map=feature_map_new, optimizer=COBYLA())

# Step 3: Train the VQR model
vqr.fit(X_train_reduced, y_train)

In [ ]:
# Predictions
y_pred_train = vqr.predict(X_train_reduced)
y_pred_test = vqr.predict(X_test_reduced)

# Evaluation metrics
mse_train = mean_squared_error(y_train, y_pred_train)
mse_test = mean_squared_error(y_test, y_pred_test)
mae_test = mean_absolute_error(y_test, y_pred_test)
r2_test = r2_score(y_test, y_pred_test)

print("Quantum SVR Train MSE:", mse_train)
print("Quantum SVR Test MSE:", mse_test)
print("Quantum SVR Test MAE:", mae_test)
print("Quantum SVR Test R2 Score:", r2_test)

## Designing High Tc Superconductors using surrogate QML model based QAOA optimisation

After we have our QML model, we follow a strategy to generate new compound rows to be fed to the QML model for prediction of Tc. This is done by creating new rows with QAOA having different compositional parameters, feeding them to QML model for prediction, and using it as a cost function (maximization, in our case) to drive QAOA algorithm towards better candidates generation.